In [1]:
import visualization
import pyomomachsched
from shift import ShiftModel
import utility
import orders
import numpy as np
import datetime
import pandas as pd
import random


# Genetic Algorithm

In [ ]:
'''order_df = orders.get_westaflex_orders()
machines = order_df["selected_machine"].astype(int).unique()
start = datetime.datetime(2022, 2, 27, 6, 0, 0)
end = datetime.datetime(2022, 3, 4, 22, 0, 0)
last_tool = "A0"
company = order_df.attrs["company_name"]
df = order_df'''

In [2]:
original_df = orders.get_westaflex_orders()
machineslist = [1531, 1532, 1533,1534, 1535, 1536, 1537,1541, 1542, 1543]
start = datetime.datetime(2022, 2, 27, 6, 0, 0)
end = datetime.datetime(2022, 3, 4, 22, 0, 0)
original_df["duration"] = (original_df["setuptime_material"] + original_df["setuptime_coil"] + original_df["duration_machine"] + original_df["duration_manual"]).dt.seconds # am Ende überprüfen ob es klappt
    #df["chosen_machine"] = [0]*len(df)
original_df["lateness"] = [0]*len(original_df)


In [ ]:
original_df.tail(n=10)

In [ ]:
ids=[]
for i in range (len(original_df)):
    x = len(original_df)-(i+1)
    ids.append(x)


In [ ]:
dftest = original_df

In [ ]:
dftest = original_df.reset_index(drop=True)
dftest.index = ids

In [ ]:
print(ids)

In [ ]:
dftest.tail(n=10)

In [ ]:
makespan(ids)
make

In [3]:
#resetting/initialising the dataframe using orders.py
def initializing_new_df(ids):
    df = original_df.reset_index(drop=True)
    df.index = ids
    df = df.sort_index(ascending=True)
    #machines = df["selected_machine"].astype(int).unique()
    
    
    #last_tool = "A0"
    #company = df.attrs["company_name"]
    

    
    df.head(n=10)
    return df

In [4]:
def create_helperframe():
    data = {
        "machineworkload": [0] * len(machineslist),
        "Jobs" : [[],[],[],[],[],[],[],[],[],[]], # manuelle Eingabe wichtig, da sonst nur Kopien von einer List erstellt werden
       }

    helperframe = pd.DataFrame(data=data,index=machineslist)
    helperframe = helperframe.reset_index(drop=True)
    helperframe.index = machineslist
    return helperframe

In [5]:
#weise den ersten job die erste Maschine zu die frei ist (eventuell random, oder siehe notizen)

#Problem: Füge bei dem ersten Wert das Startdatum hinzu und starte bei Wert 2, ansonsten Probleme mit calculated_start

def earliest_deadline_first(ids):


    df = initializing_new_df(ids)
    
    
    #creating helperframe to determine the right machine which has to be used
    #
    helperframe = create_helperframe()

    #Ordering the jobs to the right machine from index 1 to end of df
    for i in range(len(df)): 
        machine = 0

        included_keys = df["machines"].iloc[i].split(",")
        included_keys_list = list(map(int, included_keys ))
        included_keys_tuple = tuple(list(map(int, included_keys )))



        minimum = min(helperframe.loc[included_keys_tuple,:]["machineworkload"])#berechnet die geringst Auslastung der ausgewählten Maschinen
        machines_index = helperframe.index[helperframe["machineworkload"] == minimum].tolist() # Maschine(n) die auf Basis der minimalen Auslastung gewählt wurden.List als Ausgabe

        if len(machines_index) > 1:
            machine = random.choice(machines_index)
        else:
            machine = machines_index[0]


        current_workload = int(helperframe.loc[machine]["machineworkload"])


        helperframe.at[machine, "machineworkload"] += df.iloc[i,:]["duration"]
        df.at[i,"selected_machine"] = machine

        if len(helperframe.loc[machine]["Jobs"])>0:
            #hier müssen wir schauen welcher job zuletzt auf der maschine lief und die endzeit als startzeit wählen
            machine_jobs = helperframe.loc[machine]["Jobs"] #Anzeigen der bisherigen Jobs
            last_job_id = machine_jobs[-1] # Die ID des letzten Jobs ausgeben, um den neuen an das Ende des alten anzuschließen
            df.at[i, "calculated_start"] = df.loc[last_job_id]["calculated_end"] + datetime.timedelta(seconds=current_workload)
        else:
            df.at[i, "calculated_start"] = start
        
        
        df.at[i, "calculated_end"] = df.loc[i]["calculated_start"] + datetime.timedelta(seconds=int(df.loc[i]["duration"]))
        df.at[i, "lateness"] = df.loc[i]["calculated_end"] - df.loc[i]["deadline"]

        helperframe.at[machine,"Jobs"].append(i)# = helperframe.loc[machine]["Jobs"].append(i)



    #
    #caclulate makespan
    #bei jeden job, dessen calculated_end > end ist, die duration auf 0 setzen, dann alle durations addieren, oder ist makespan die anzahl der aufträge?

    return df

In [6]:
def average_lateness(ids):
    df = earliest_deadline_first(ids)
    #Average lateness
    lateness_seconds = pd.to_timedelta(df["lateness"]).dt.total_seconds()
    #lösche Negative Einträge zu 0:
    lateness_seconds = lateness_seconds.apply(lambda x: x if x>=0 else 0)
    total_lateness = lateness_seconds.sum()
    average_lateness = total_lateness / len(df)
    return average_lateness

In [7]:
#makespan

#ich vermute es ist am sinnvollsten duration zu addieren, da vermutlich die jobs mit der längsten duration auch am wertvollsten sind
#ansonsten einfach über den helperframe die länge der joblisten nutzen
def makespan(ids):
    df = earliest_deadline_first(ids)
    late_jobs = df
    duration = df["duration"].to_frame()
    calc_end = df["calculated_end"].to_frame()


    zero_entries = df

    for i in range (len(df)):
        if end<zero_entries.loc[i]["calculated_end"]:

            zero_entries.at[i,"duration"] = 0

    makespan = zero_entries["duration"].sum()
    return makespan

#duration.apply(lambda x: 0 if end < df.loc[x]["calculated_end"] else x) dies könnte schneller ansatz sein, aber vielleicht nicht möglich
#die Ausgabe ist in Sekunden



In [ ]:
'''
to do:
 - die oberen Zeilen vernünftig in Funktionen packen vermutlich makespan und average lateness als 2 verschiedene
   objective functions nutzen und bei jeder neuen Generation muss die obere Formatierung benutzt werden, damit man die
   beiden functions anwenden kann
 - Einmal schauen ob man Pymoo nutzen kann und falls nicht
 - das untere Template nutzen, oder einmal bei chatgpt ein Template nutzen
 - Freitag 7,5h arbeiten, dann sind 20 Stunden voll diese Woche

'''

In [ ]:
from pymoo.operators.crossover.pntx import TwoPointCrossover
from pymoo.operators.sampling.rnd import PermutationRandomSampling
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.mutation.inversion import InversionMutation
from pymoo.operators.crossover.ox import OrderCrossover
from pymoo.core.problem import ElementwiseProblem
from pymoo.operators.crossover.sbx import SBX
import time

id_init = list(original_df.index)
id_init_array=np.array(id_init)
starttime = time.time()
print("Start")

class MyProblem(ElementwiseProblem):

    def __init__(self):
        super().__init__(
                         n_var=190,
                         n_obj=2,
                         n_ieq_constr=0, # constrains auf zero if we dont need them 
                         )

    def _evaluate(self, x, out, *args, **kwargs):
        f1 = -abs(makespan(x))
        f2 = average_lateness(x)


        out["F"] = [f1, f2]

'''def obj1(x):
    return -abs(makespan(x))

def obj2(x):
    
    return average_lateness(x)

problem = Problem(n_var=190, n_obj=2, objectives=[obj1, obj2])
'''

problem = MyProblem()
algorithm = NSGA2(
                    pop_size=190,
                    pop=id_init,
                    mutation=InversionMutation(),
                    #crossover=SBX(prob=0.9, eta=15),
                    crossover=OrderCrossover(),
                    sampling=PermutationRandomSampling(),
                    
                 )

from pymoo.termination import get_termination

termination = get_termination("n_gen", 3)

from pymoo.optimize import minimize

res = minimize(problem,
               algorithm,
               termination,
               seed=1,
               display=None,
              )

X = res.X
F = res.F

endtime = time.time()
print(endtime - starttime)

hello


In [9]:
print(res.F)

[[-1216408.          1360154.06315789]
 [-1260602.          1579523.12105263]
 [-1266890.          1624086.83157895]
 [-1130578.          1300758.2       ]
 [-1423931.          2221572.41052632]
 [-1289465.          1697991.8       ]
 [-1128910.          1167220.55789474]
 [-1394656.          1900055.37894737]
 [-1323893.          1833224.48947368]
 [-1319824.          1804216.91052632]]


In [10]:
print(X)

[[ 12 122  46 ... 182 165  77]
 [ 13 183 174 ...  39 169  31]
 [108  95 168 ...  19   2 136]
 ...
 [ 55 165  52 ... 159 136 137]
 [189 111  89 ... 133 138 121]
 [ 14  99  79 ... 126 142  49]]


In [ ]:
#Welches X gehört zu welchem F?

In [ ]:
from pymoo.problems.single.traveling_salesman import create_random_tsp_problem
problem = create_random_tsp_problem(30, 100, seed=1)
print(problem)

In [ ]:
print(end)
print(zero_entries.loc[1]["calculated_end"])
print(end<zero_entries.loc[1]["calculated_end"])

In [ ]:
df.head(n=20)

In [ ]:
#uniqeness ohne Berücksichtigung der Maschine. Erweitere auf alle Maschinen
endtime = dt.datetime(2022, 3, 4, 22)
starttimeList = []
endtimeList = []
for i in range(len(df)): 

    if endtime <= df["input"].iloc[i]: 
        starttime = df["input"].iloc[i]
        print("start time: " + str(starttime))
        starttimeList.append(starttime)

        endtime = df["input"].iloc[i] + df["duration"].iloc[i]
        print("end time:   " + str(endtime))
        endtimeList.append(endtime)

    else:   
        starttime = endtime
        print("start time: " + str(starttime))
        starttimeList.append(starttime)

        endtime = starttime + df["duration"].iloc[i]
        print("end time:   " + str(endtime))
        endtimeList.append(endtime)


In [ ]:
# Für jede Maschine
for machine in machines:
    df_machine = order_df[order_df["selected_machine"].astype(int) == machine]
    timestamp = start
    # Entsprechend der Reihenfolge timestamps berechnen
    for index, row in df_machine.iterrows():
        order_num = row["job"]
        shift = row["shift"]

       # TODO: What about already running jobs? Or jobs that are finished?
        if timestamp < row["order_release"]:
            timestamp = row["order_release"]
            # Adjust timestamp to next shift start
        shifts = ShiftModel(company, shift, timestamp)
        timestamp = shifts.get_earliest_time(timestamp)

        order_df.loc[order_df["job"] == order_num, ["calculated_start"]] = timestamp
        tool = row["tool"]
        setup_time = row["setup_time"] if str(tool).casefold() != str(last_tool).casefold() else datetime.timedelta(0)
        order_df.loc[order_df["job"] == order_num, ["calculated_setup_time"]] = setup_time
        prod_time = row["duration_machine"]
        if isinstance(prod_time, datetime.timedelta):
            prod_time = prod_time.total_seconds() / 60
        if isinstance(setup_time, datetime.timedelta):
            setup_time = setup_time.total_seconds() / 60
        runtime = prod_time - 17 + setup_time + 2
        timestamp = utility.calculate_end_time(
            start=timestamp, duration=runtime, company=company, shift=shift
        )
        order_num = row["job"]
        order_df.loc[order_df["job"] == order_num, ["calculated_end"]] = timestamp
        last_tool = tool


In [ ]:
order_df["duration"] = df["setuptime_material"] + df[""]
df.head(n=10)

In [ ]:
starttime = start
df=order_df
if not isinstance(starttime, datetime.datetime):
    raise ValueError(
            f"starttime parameter needs to be datetime object. Given: {type(starttime)}"
        )
    # Assert that all necessary columns are in the dataframe
    necessary_columns = set(
        [
            "order_release",
            "machines",
            "duration_machine",
            "setup_time",
            "tool",
            "shift",
            "deadline",
        ]
    )
    if not necessary_columns.issubset(df.columns):
        raise ValueError(
            "Given DataFrame does not have all the necessary columns. "
            + f"Missing columns: {necessary_columns - df.columns}"
        )
    df["order_release"] = df["order_release"].apply(
        lambda x: starttime if x < starttime else x
    )

    machines = set(
        [
            machine
            for machine_list in df["machines"].tolist()
            for machine in machine_list.split(",")
            if machine != ""
        ]
    )
    company = df.attrs["company_name"]
    if not company:
        raise ValueError(
            "Given DataFrame does not contain the company name as attribute."
        )
    amount_of_jobs = len(df.index)
    jobs = range(amount_of_jobs)
    # All datetime values throughout this function need to be
    # converted to seconds, since pyomo does not calculate with datetimes.
    # Subtract setuptime from durations, since they are already part of that
    durations = {
        j: df.iloc[j]["duration_machine"].total_seconds()
        - df.iloc[j]["setup_time"].total_seconds()
        for j in jobs
    }
    first_release = df["order_release"].min().to_pydatetime()



In [ ]:
df.head(n=10)

In [ ]:
def objectiveFunctionDeadline(x):
    #Vorzeiteger Abschluss des Auftrags wird nicht positiver gewertet als genaues erreichen der Deadline
    def posfunction(x):
        if x<0:
            x=0
    return x
    amount_of_jobs = len(df.index)
    #implement a deadline missing row
    df["missed_deadline"] = (df["deadline"]-df["calculated_end"]).dt.seconds
    #if we hit a deadline the negative value should be counted as 0 for now it seems like we never miss a deadline.
    #this is suspicious and should be further investigated
    df["missed_deadline"] = df.apply(lambda row: posfunction(row['missed_deadline']), axis=1)

    return (1/amount_of_jobs)*(np.sum(df["missed_deadline"].to_numpy()))


In [ ]:
def objectiveFunctionMaschine(x):
    

In [ ]:
#objective function
def posfunction(x):
    if x<0:
        x=0
        print("yes")
    return x
amount_of_jobs = len(df.index)
#implement a deadline missing row
df["missed_deadline"] = (df["deadline"]-df["calculated_end"]).dt.seconds
#if we hit a deadline the negative value should be counted as 0 for now it seems like we never miss a deadline.
#this is suspicious and should be further investigated
df["missed_deadline"] = df.apply(lambda row: posfunction(row['missed_deadline']), axis=1)

display(df["missed_deadline"])
#display(df)
objective_function = (1/amount_of_jobs)*(np.sum(df["missed_deadline"].to_numpy()))

print(objective_function)
print((df['missed_deadline'] == 0).sum())

## Pyomo naive genetic

In [ ]:
#Genetic Algorithm Template

from random import choices, randint, randrange, random
from typing import List, Optional, Callable, Tuple

'''Genome = List[int]
Population = List[Genome]
PopulateFunc = Callable[[], Population]
FitnessFunc = Callable[[Genome], int]
SelectionFunc = Callable[[Population, FitnessFunc], Tuple[Genome, Genome]]
CrossoverFunc = Callable[[Genome, Genome], Tuple[Genome, Genome]]
MutationFunc = Callable[[Genome], Genome]
PrinterFunc = Callable[[Population, int, FitnessFunc], None]
'''


def generate_genome(length):
    return choices([0, 1], k=length)


def generate_population(size, genome_length):
    return [generate_genome(genome_length) for _ in range(size)]


def single_point_crossover(a, b):
    if len(a) != len(b):
        raise ValueError("Genomes a and b must be of same length")

    length = len(a)
    if length < 2:
        return a, b

    p = randint(1, length - 1)
    return a[0:p] + b[p:], b[0:p] + a[p:]


def mutation(genome, num = 1, probability= 0.5):
    for _ in range(num):
        index = randrange(len(genome))
        genome[index] = genome[index] if random() > probability else abs(genome[index] - 1)
    return genome


def population_fitness(population, fitness_func):
    return sum([fitness_func(genome) for genome in population])


def selection_pair(population, fitness_func):
    return choices(
        population=population,
        weights=[fitness_func(gene) for gene in population],
        k=2
    )


def sort_population(population, fitness_func):
    return sorted(population, key=fitness_func, reverse=True)


def genome_to_string(genome):
    return "".join(map(str, genome))


def print_stats(population, generation_id, fitness_func):
    print("GENERATION %02d" % generation_id)
    print("=============")
    print("Population: [%s]" % ", ".join([genome_to_string(gene) for gene in population]))
    print("Avg. Fitness: %f" % (population_fitness(population, fitness_func) / len(population)))
    sorted_population = sort_population(population, fitness_func)
    print(
        "Best: %s (%f)" % (genome_to_string(sorted_population[0]), fitness_func(sorted_population[0])))
    print("Worst: %s (%f)" % (genome_to_string(sorted_population[-1]),
                              fitness_func(sorted_population[-1])))
    print("")

    return sorted_population[0]


def run_evolution(
        populate_func,
        fitness_func,
        fitness_limit,
        selection_func=selection_pair,
        crossover_func=single_point_crossover,
        mutation_func=mutation,
        generation_limit= 100,
        printer= None
    ):
    population = populate_func()

    for i in range(generation_limit):
        population = sorted(population, key=lambda genome: fitness_func(genome), reverse=True)

        if printer is not None:
            printer(population, i, fitness_func)

        if fitness_func(population[0]) >= fitness_limit:
            break

        next_generation = population[0:2]

        for j in range(int(len(population) / 2) - 1):
            parents = selection_func(population, fitness_func)
            offspring_a, offspring_b = crossover_func(parents[0], parents[1])
            offspring_a = mutation_func(offspring_a)
            offspring_b = mutation_func(offspring_b)
            next_generation += [offspring_a, offspring_b]

        population = next_generation

    return population, i

In [ ]:
print(type(df["calculated_end"]))

In [ ]:
def naive_termination(order_df, start, last_tool):
    """
    Calculates a simple termination from the given orders and returns it.

    Parameters
    ----------
    order_df: dataframe
        Orders in a dataframe containing the columns:
            machine, job, shift, order_release, duration_machine,
            setup_time
        Overwrites the values of following columns:
            calculated_start, calculated_end.
    start: datetime
        The start time of the naive termination calculation.
    last_tool: str
        The last tool name that has been used in the last job before the
        termination.

    Returns
    -------
    dataframe
        The orders with overwritten calculated_start and calculated_end.
    """
    machines = order_df["selected_machine"].astype(int).unique()
    print(machines)
    company = order_df.attrs["company_name"]
    # Für jede Maschine
    for machine in machines:
        df_machine = order_df[order_df["selected_machine"].astype(int) == machine]
        timestamp = start
        # Entsprechend der Reihenfolge timestamps berechnen
        for index, row in df_machine.iterrows():
            order_num = row["job"]
            shift = row["shift"]

            # TODO: What about already running jobs? Or jobs that are finished?
            if timestamp < row["order_release"]:
                timestamp = row["order_release"]
            # Adjust timestamp to next shift start
            shifts = ShiftModel(company, shift, timestamp)
            timestamp = shifts.get_earliest_time(timestamp)

            order_df.loc[order_df["job"] == order_num, ["calculated_start"]] = timestamp
            tool = row["tool"]
            setup_time = row["setup_time"] if tool.casefold() != last_tool.casefold() else datetime.timedelta(0)
            order_df.loc[order_df["job"] == order_num, ["calculated_setup_time"]] = setup_time
            prod_time = row["duration_machine"]
            if isinstance(prod_time, datetime.timedelta):
                prod_time = prod_time.total_seconds() / 60
            if isinstance(setup_time, datetime.timedelta):
                setup_time = setup_time.total_seconds() / 60
            runtime = prod_time - 17 + setup_time + 2
            timestamp = utility.calculate_end_time(
                start=timestamp, duration=runtime, company=company, shift=shift
            )
            order_num = row["job"]
            order_df.loc[order_df["job"] == order_num, ["calculated_end"]] = timestamp
            last_tool = tool
    return order_df


def combine_orders(order_df, start):
    """
    Combines orders with the same item properties.
    """
    # TODO: FINISH THIS FUNCTION


if __name__ == "__main__":
    # Debugging
    df = orders.get_westaflex_orders()
    df.drop(index=df.index[:180], axis=0, inplace=True)
    '''print(
        df[
            [
                "order_release",
                "machines",
                "selected_machine",
                "shift",
                "duration_machine",
                "calculated_start",
                "calculated_end",
                "setup_time",
            ]
        ]
    )'''
    df = naive_termination(df, datetime.datetime(2022, 2, 27, 6, 0, 0), "A0")
    print(
        df[
            [
                "order_release",
                "machines",
                "selected_machine",
                "shift",
                "duration_machine",
                "calculated_start",
                "calculated_end",
                "setup_time",
            ]
        ]
    )
    visualization.gantt(df)
    '''df = pyomomachsched.opt_schedule(df, datetime.datetime(2022, 1, 2, 0, 0, 0))
    with pd.option_context("display.max_rows", None, "display.max_columns", None):
        print(
            df[
                [
                    "order_release",
                    "tool",
                    "machines",
                    "selected_machine",
                    "shift",
                    "duration_machine",
                    "setup_time",
                    "calculated_start",
                    "calculated_end",
                ]
            ]
        )
    df.to_pickle("./pyomo_df.pkl")
    df.to_excel("./pyomo_df.xlsx")
    df.to_csv("./pyomo_df.csv")
    visualization.gantt(df)'''

